In [1]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [2]:
#importing requisite modules
import pdfplumber
import re

import pandas as pd
from collections import namedtuple
import csv

In [3]:
#PART 1: READING THE eTRR AND EXTRACTING THE SUMMARY PAGE DATA (in the original layout)



Summary=[] #empty list decalred to store the summary of tyhe rake
siding_charges=0.0
AB=[]
#reading the pdf document using pdfplumer
with pdfplumber.open(r'KSG-MPCT21121529_RK-47_01122021.pdf') as pdf: 
    page = pdf.pages[0] #extracting only the first page for the summary of the rake
    text= page.extract_text() #extaracting all text from the page
    for line in text.split('\n'): #spliting the text in to seperate lines at every new line character
        Z=line.split()
        #print (Z)
        AB.append(Z)
DF= pd.DataFrame(AB)

DF.head(10)
DF.to_csv("SummaryETRR.csv")

In [4]:
#PART2: FIND THE CHARGES FROM THE SUMMARY SHEET


Summary=[] #empty list decalred to store the summary of the rake
siding_charges=0.0

#reading the pdf document using pdfplumer
with pdfplumber.open(r'KSG-MPCT21121529_RK-47_01122021.pdf') as pdf: 
    page = pdf.pages[0] #extracting only the first page for the summary of the rake
    text= page.extract_text() #extaracting all text from the page
    for line in text.split('\n'): #spliting the text in to seperate lines at every new line character
        #print (line) 
        Summary.append(line) # adding each line to the summary list
        if line.startswith('SD'): #to extarct the siding charges levied in the e-TRR as string
            siding_charges= float(line[3:])  #typecasting string to float to ease calculation and storing in the variable
        elif line.startswith('DS'):  #to extarct the development surcharges levied in the e-TRR as string
            dev_surcharge= float(line[3:]) #typecasting string to float to ease calculation and storing in the variable
    
df_summary=pd.DataFrame(Summary) #Creating a dataframe for the summary data
#print (Summary)
print ("Siding Charges = ",siding_charges)
print ("Development Charges= ",dev_surcharge)
name= df_summary[0][1][:17]

print (name)
#df_summary

Siding Charges =  0.0
Development Charges=  69397.9
RR No: 262003367 


In [5]:
#PART 3: CREATING A STRUCTURED SUMMARY OF THE FREIGHT BY FORMING A DATAFRAME AND FROM THE DATAFRAME MAKING A CSV FILE

#Reading rake and container data from the pdf and storing in structured data form

FREIGHT_DETAILS=[]  #empty list declared to store all details related to freight and container
Nxt=[]
Nxt1=[]
#named tuple declared to store in the same pattern the data extracted from the pdf
Line= namedtuple('Line','SNO OWNR WGON_TYPE WGON_NO CNTR_NO CNTR_SIZE CNTR_LE CNTR_POS CNTR_CMDT TRFC_TYPE CMDT_CODE COMP_TEU CHRG_TEU GWT FT')
c=1
d=1
e=1
f=1
#declaring regular expression compile pattern to search for freight data. pattern obtained by visually checking the data
freight_line_re= re.compile((r'(\d) (.*) (.*) (\d{4}) (\d{1}) [1-2]'))
#freight_line_2_re= re.compile ((r'(.{1}) (\d{5})'))
Second_line= re.compile (r'" [A-Z] "" (\d{5})')
Third_line= re.compile (r'[0-9]')
#reading the entire document
with pdfplumber.open(r'KSG-MPCT21121529_RK-47_01122021.pdf') as pdf:
    for page in pdf.pages:
        
        text= page.extract_text() #extracting text from document
        for line in text.split('\n'):              #spliting text at each new line
                c=c+1
                data= freight_line_re.search(line)  #regular expression search to match with the previously declared data pattern
                #if search successful, then storing the requisite data into the desired format
                if data:
                    in_lines = True
                    print (line)             
                    X=line.split()
                    #print (X)
                    if X[6] is 'E':
                        X.insert (8,"  ")
                        #print (X)
                    
                        
                        
                    SNO,OWNR,WGON_TYPE,WGON_NO,CNTR_NO,CNTR_SIZE,CNTR_LE,CNTR_POS,CNTR_CMDT,TRFC_TYPE,CMDT_CODE,COMP_TEU,CHRG_TEU,GWT,FT=X
                    #print (X)
                    line_info= Line(SNO,OWNR,WGON_TYPE,WGON_NO,CNTR_NO,CNTR_SIZE,CNTR_LE,CNTR_POS,CNTR_CMDT,TRFC_TYPE,CMDT_CODE,COMP_TEU,CHRG_TEU,GWT,FT)
                    #print (line_info)
                    #print (line_info[3])
                    
                    #adding each valid data to the list
                    FREIGHT_DETAILS.append(line_info)
                    e=c
                    
                    
                data1= Second_line.match(line)
                d=d+1
                if d==e+1 and d>2:
                    #print ("data1", data1)
                    #print (line)
                    #print(line[0],line[2:])
                    Nxt.append(line)
                    
                         
                data2= Third_line.search(line)
                f=f+1
                if f==e+2 and f>3:
                    #print ("data2", data2)
                    #print (line)
                    #print(e,f)
                    Nxt1.append(line)
               
    
df0=pd.DataFrame(FREIGHT_DETAILS)
dfx=pd.DataFrame(Nxt)
dfx1=pd.DataFrame(Nxt1)
for i in df0.index:
    df0.WGON_TYPE[i]= df0.WGON_TYPE[i]+dfx[0][i][0]
    df0.WGON_NO[i]= df0.WGON_NO[i]+dfx[0][i][2:]+dfx1[0][i]
df0.to_csv("Final_data_ETRR.csv")
df0



<>:33: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:33: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-5-95e83c7527ee>:33: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if X[6] is 'E':


1 ADAL BLCA 61320 FSCU9205628 40 L LD NIGT EXIM 0952 2 1.8 26.69 22920.84
2 ADAL BLCA 61320 MEDU4450246 40 L UD NIGT EXIM 0952 2 1.8 25.17 11460.42
3 ADAL BLCB 62320 BAXU2663031 20 L LD R EXIM 0952 1 1 22.91 15550.55
45 ADAL BLCB 62320 OOLU8725808 40 L UD NIGT EXIM 0952 2 1.8 5.72 8942.45
4 ADAL BLCB 62320 TCKU2973647 20 L LD NIGT EXIM 0952 1 1 18.81 12733.80
47 ADAL BLCB 62320 BMOU2367189 20 L LD NIGT EXIM 0952 1 1 29.32 17450.55
48 ADAL BLCB 62320 HASU4937095 40 L UD NIGT EXIM 0952 2 1.8 12.62 8942.45
46 ADAL BLCB 62320 MEDU5798067 20 L LD NIGT EXIM 0952 1 1 17.58 12733.80
49 ADAL BLCB 62320 SEGU6099435 40 L LD NIGT EXIM 0952 2 1.8 23.84 22920.84
50 ADAL BLCB 62320 ZCSU7994343 40 L UD NIGT EXIM 0952 2 1.8 14.02 8942.45
52 ADAL BLCA 61320 TCLU5706750 40 L UD NIGT EXIM 0952 2 1.8 23.72 11460.42
51 ADAL BLCA 61320 TRHU4965567 40 L LD NIGT EXIM 0952 2 1.8 25.25 22920.84
53 ADIL BLCA 61321 BSIU2585268 20 L LD NIGT EXIM 0952 1 1 27.2 17450.55
55 ADIL BLCA 61321 CAXU6891154 20 L LD NIGT EXI

,SNO,OWNR,WGON_TYPE,WGON_NO,CNTR_NO,CNTR_SIZE,CNTR_LE,CNTR_POS,CNTR_CMDT,TRFC_TYPE,CMDT_CODE,COMP_TEU,CHRG_TEU,GWT,FT
0,1,ADAL,BLCAM,61320800374,FSCU9205628,40,L,LD,NIGT,EXIM,0952,2,1.8,26.69,22920.84
1,2,ADAL,BLCAM,61320800374,MEDU4450246,40,L,UD,NIGT,EXIM,0952,2,1.8,25.17,11460.42
2,3,ADAL,BLCBM,62320800630,BAXU2663031,20,L,LD,R,EXIM,0952,1,1,22.91,15550.55
3,45,ADAL,BLCBM,62320800630,OOLU8725808,40,L,UD,NIGT,EXIM,0952,2,1.8,5.72,8942.45
4,4,ADAL,BLCBM,62320800630,TCKU2973647,20,L,LD,NIGT,EXIM,0952,1,1,18.81,12733.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,92,ADIL,BLCBM,62321910444,TCNU1359264,40,L,LD,NIGT,EXIM,0952,2,1.8,14.18,17884.89
93,94,ADIL,BLCBM,62321910451,BMOU4235411,40,L,LD,NIGT,EXIM,0952,2,1.8,22.12,22920.84
94,95,ADIL,BLCBM,62321910451,MSDU8132106,40,L,UD,NIGT,EXIM,0952,2,1.8,14.05,8942.45
95,97,ADIL,BLCAM,61321910300,FSCU6468620,40,L,UD,NIGT,EXIM,0952,2,1.8,8.22,8942.45


In [12]:

#SPLITING THE CHARGES WITH IB TRAIN AN RECALCULATING THE CHARGES


DS=[] #to store the development surcharges at container level
SD=[] #to store the siding charges at container level
GST=[] #to store the GST applicable at container level
total=[] # to store the total charges at container level
gstt=0.0 #to store the total GST on all containers
sdt=0.0 #to store the total Siding charges on all container
dst=0.0 #to store the total Development surcharges on all container
total1=0.0 #to store the total freight charges on all container
total_freight=0.0 #to store the total freight including all applicable additional charges on all container
taxable_amt=0.0 #to store the taxable amount on which GST is applicable
a=0 #to calculate the total TEUs
b=0
#To find the total TEU and spliting SD for th rake into half assuming that inbound rake is also having the same no. of rakes. Finding DS at container level
for i in df0.index:
    DS.append(float(df0.FT[i])*0.05)#DS is 5 percent of freight charges
    a=a + int(df0.COMP_TEU[i])
    b=b + float(df0.CHRG_TEU[i])
print("Total TEUs=", a)
SD_TEU= siding_charges/(2*b) #assuming the IB rake has equal no. of TEUs
print('Siding Charges per TEU=', SD_TEU)

for i in df0.index:
    SD.append(SD_TEU*float(df0.COMP_TEU[i]))
df0['DS']=DS
df0['SD']=SD

#checking for agri products(assuming 'R' to be comodity type for agr products) for applicability of GST
for i in df0.index:
    if df0.CNTR_CMDT[i] == "R":
        GST.append(0.0) #GST is nil for agri products
              
    else:
        taxable_amt= taxable_amt + (float(df0.FT[i])+float(df0.DS[i]))
        GST.append(0.05 *(float(df0.FT[i])+float(df0.DS[i]))) #GST is 5percent of freight and developmental surcharges
df0['GST']=GST
for i in df0.index:
    total.append(float(df0.GST[i])+(float(df0.FT[i])+float(df0.DS[i])+float (df0.SD[i])))
df0['Total']=total

#finding the each charge categry total 
for i in df0.index:
    total_freight=total_freight  + float(df0.Total[i])
    total1= total1 + float(df0.FT[i])
    gstt= gstt+ float(df0.GST[i])
    dst= dst + float (df0.DS[i])
    sdt=sdt+ float(df0.SD[i])
print ("freight=",total1)
print ("Total SD =", sdt)
print ("Total DS =",dst)
print ("total GST =", gstt)
print ("taxable amount =",taxable_amt)


print ("total freight=", total_freight)
df0.to_csv ('Recalculated freight.csv')
df0

Total TEUs= 176
Siding Charges per TEU= 0.0
freight= 1387958.2099999988
Total SD = 0.0
Total DS = 69397.9105
total GST = 72051.40215000008
taxable amount = 1441028.043
total freight= 1529407.522650001


,SNO,OWNR,WGON_TYPE,WGON_NO,CNTR_NO,CNTR_SIZE,CNTR_LE,CNTR_POS,CNTR_CMDT,TRFC_TYPE,CMDT_CODE,COMP_TEU,CHRG_TEU,GWT,FT,DS,SD,GST,Total
0,1,ADAL,BLCAM,61320800374,FSCU9205628,40,L,LD,NIGT,EXIM,0952,2,1.8,26.69,22920.84,1146.0420,0.0,1203.344100,25270.226100
1,2,ADAL,BLCAM,61320800374,MEDU4450246,40,L,UD,NIGT,EXIM,0952,2,1.8,25.17,11460.42,573.0210,0.0,601.672050,12635.113050
2,3,ADAL,BLCBM,62320800630,BAXU2663031,20,L,LD,R,EXIM,0952,1,1,22.91,15550.55,777.5275,0.0,0.000000,16328.077500
3,45,ADAL,BLCBM,62320800630,OOLU8725808,40,L,UD,NIGT,EXIM,0952,2,1.8,5.72,8942.45,447.1225,0.0,469.478625,9859.051125
4,4,ADAL,BLCBM,62320800630,TCKU2973647,20,L,LD,NIGT,EXIM,0952,1,1,18.81,12733.80,636.6900,0.0,668.524500,14039.014500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,92,ADIL,BLCBM,62321910444,TCNU1359264,40,L,LD,NIGT,EXIM,0952,2,1.8,14.18,17884.89,894.2445,0.0,938.956725,19718.091225
93,94,ADIL,BLCBM,62321910451,BMOU4235411,40,L,LD,NIGT,EXIM,0952,2,1.8,22.12,22920.84,1146.0420,0.0,1203.344100,25270.226100
94,95,ADIL,BLCBM,62321910451,MSDU8132106,40,L,UD,NIGT,EXIM,0952,2,1.8,14.05,8942.45,447.1225,0.0,469.478625,9859.051125
95,97,ADIL,BLCAM,61321910300,FSCU6468620,40,L,UD,NIGT,EXIM,0952,2,1.8,8.22,8942.45,447.1225,0.0,469.478625,9859.051125
